In [60]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
import os
import json

# Data Wrangling
import h5py
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import glob
import flammkuchen as fl
import shutil

import seaborn as sns
import matplotlib as mpl

### Custom functions

In [62]:
# from feature_extractuion_helpers import smooth_trace
# from megabouts_helper import labels_cat

from megabouts.utils import (
    bouts_category_name,
    bouts_category_name_short,
    bouts_category_color,
    cmp_bouts,
)

In [63]:
def flatten(l):
    return [item for sublist in l for item in sublist]

## pool data 

In [75]:
master_path = Path(r"\\portulab.synology.me\data\Kata\Data\230307_visstim_2D") #rectangular arena # start from fish 1
# master_path = Path(r"\\portulab.synology.me\data\Kata\Data\22042024_visstim_2D_round")
# master_path = Path(r"\\portulab.synology.me\data\Kata\Data\22042024_visstim_2D_2") #rectangular arena
# master_path = Path(r"\\portulab.synology.me\data\Kata\Data\13052024_visstim_2D_round")
# master_path = Path(r"\\portulab.synology.me\data\Kata\Data\14052024_visstim_2D_round")

fish_paths = list(master_path.glob('*f[0-9]*'))
fish_paths, len(fish_paths)

([WindowsPath('//portulab.synology.me/data/Kata/Data/230307_visstim_2D/105234_f0'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/230307_visstim_2D/105646_f0'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/230307_visstim_2D/110659_f0'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/230307_visstim_2D/111701_f0'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/230307_visstim_2D/112901_f1'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/230307_visstim_2D/113916_f2'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/230307_visstim_2D/114946_f2'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/230307_visstim_2D/120328_f3'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/230307_visstim_2D/120350_f3'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/230307_visstim_2D/123953_f4'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/230307_visstim_2D/125639_f5'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/230307_visstim_2D/13083

In [76]:
exp_name = Path(fish_paths[0]).parts[-2]
exp_name

'230307_visstim_2D'

In [77]:
out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\230307_visstim_2D_")
# out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\22042024_visstim_2D_round_")
# out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\22042024_visstim_2D_2_")
# out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\13052024_visstim_2D_round_")
# out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\14052024_visstim_2D_round_")

### Tensor pooling

In [78]:
tensor_path = Path(r'\\portulab.synology.me\data\Kata\Processed_Data\tensor_data\tensors')

In [79]:
tensors_ = []
indices_ = []

for ind, fish in enumerate(fish_paths):
    print ('Working on {}'.format(fish.name))
    fish_id = fish.name
    
    try:
        tensor = fl.load(Path(out_path/'{}_tensor.h5'.format(fish_id)))
        print (tensor.shape)
        tensors_.append(tensor)
    except OSError:
        pass
    


Working on 105234_f0
Working on 105646_f0
(47, 3, 50)
Working on 110659_f0
(68, 3, 50)
Working on 111701_f0
(6, 3, 50)
Working on 112901_f1
(189, 3, 50)
Working on 113916_f2
(212, 3, 50)
Working on 114946_f2
(243, 3, 50)
Working on 120328_f3
(361, 3, 50)
Working on 120350_f3
(242, 3, 50)
Working on 123953_f4
(64, 3, 50)
Working on 125639_f5
(213, 3, 50)
Working on 130839_f5
(362, 3, 50)
Working on 132040_f6
(315, 3, 50)
Working on 133118_f6
(354, 3, 50)


In [80]:
### stack tensors per experiment
# Concatenate tensors along the first axis
big_tensor = np.concatenate(tensors_, axis=0)

# Output the shape of the resulting tensor
print(big_tensor.shape)
fl.save(tensor_path/ '{}_tensor_norm.h5'.format(exp_name), big_tensor)

(2676, 3, 50)


In [83]:
t_paths = list(tensor_path.glob('*tensor_norm.h5*'))
t_paths, len(t_paths)
tensors_pooled =[]

for ind, path in enumerate(t_paths):
    print ('Working on {}'.format(path.name))
    try:
        tensor = fl.load(path)
        print (tensor.shape)
        tensors_pooled.append(tensor)
    except OSError:
        pass


Working on 13052024_visstim_2D_round_tensor_norm.h5
(2228, 3, 50)
Working on 14052024_visstim_2D_round_tensor_norm.h5
(2509, 3, 50)
Working on 22042024_visstim_2D_2_tensor_norm.h5
(2228, 3, 50)
Working on 22042024_visstim_2D_round_tensor_norm.h5
(10047, 3, 50)
Working on 230307_visstim_2D_tensor_norm.h5
(2676, 3, 50)


In [84]:
### stack tensors per experiment
# Concatenate tensors along the first axis
pooled_tensor = np.concatenate(tensors_pooled, axis=0)

# Output the shape of the resulting tensor
print(pooled_tensor.shape)
fl.save(Path(r'\\portulab.synology.me\data\Kata\Processed_Data\tensor_data')/ 'pooled_tensor_norm.h5', pooled_tensor)

(19688, 3, 50)
